<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 - Food vision milestone Project 1

In the previous chapter (**transfer learning part 3: scaling up**) we build Food Vision mini: a transfer learning model which beat the original results of the [Food 101 paper](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/) with only 10% of the data.

Now, what would happen if we used all the data ?

Well, that's we're going to find out in this notebook!

We're going to be building Food Vision Big, using all of the data from the Food101 dataset.

This time **we've got the goal of beating [DeepFood](https://www.researchgate.net/publication/304163308_DeepFood_Deep_Learning-Based_Food_Image_Recognition_for_Computer-Aided_Dietary_Assessment), a 2016 paper which used a Convolutional Neural Network trained for 2-3 days to achieve 77.4% top-1 accuracy.

> **Note :** **Top-1 accuracy** means "accuracy for the top softmax activation value output by the model" (because softmax outputs a value for every class, but top-1 means only the highest one is evaluated). **Top-5 accuracy** means "accuracy for the top 5 softmax activation values output by the model", in other words, did the true label appear in the top 5 activation values? Top-5 accuracy scores are usually noticeably higher than top-1.


<table>
    <thead>
        <tr>
            <th></th>
            <th>🍔👁 Food Vision Big™</th>
            <th>🍔👁 Food Vision mini</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Dataset source</td>
            <td>TensorFlow Datasets</td>
            <td>Preprocessed download from Kaggle</td>	
        </tr>
        <tr>
            <td>Train data</td>
            <td>75,750 images</td>
            <td>7,575 images</td>	
        </tr>
        <tr>
            <td>Test data</td>
            <td>25,250 images</td>
            <td>25,250 images</td>	
        </tr>
        <tr>
            <td>Mixed precision</td>
            <td>Yes</td>
            <td>No</td>	
        </tr>
        <tr>
            <td>Data loading</td>
            <td>Perform tf.data API</td>
            <td>TensorFlow pre-built function</td>	
        </tr>
        <tr>
            <td>Target results</td>
            <td>77,4% top-1 accuracy</td>
            <td>50,76% top-1 accuracy</td>	
        </tr>
    </tbody>
</table>

Alongside attempting to beat the DeepFood paper, we're going to learn about two methods to significantly improve the speed of our model training:
1. Prefetching
2. Mixed precision training